In [ ]:
#import moduls
import pygimli as pg
import pygimli.meshtools as mt
from pygimli.physics import ert
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#define geometry
left = 50
right = 50
depth = 50

domain = mt.createWorld(start = [-left, 0], 
                        end = [right, -depth], 
                        layers = [-1, -10],
                        worldMarker=True)

#adda a polygon
blob = mt.createPolygon([[0, -2.5],
                       [5, -2],
                       [12, -3.4], 
                       [8, -3.6],
                       [3, -3.7]], 
                       isClosed=True,
                       addNodes=8, 
                       interpolate='spline', 
                       marker = 4)

geom = domain + blob
pg.show(geom)


In [ ]:
#create synthetic electrodes
# from -15 to 15 m, we are putting 21 sensors using dipole-dipole scheme
scheme = ert.createData(elecs=np.linspace(start=-20, stop=20, num=21),
                           schemeName='wa')

# further refinement nodes at 10% distance  of electrode spacing for better accuracy 
for p in scheme.sensors():
    geom.createNode(p)
    geom.createNode(p - [0, 0.1])
    
#create mesh
mesh = mt.createMesh(geom, quality = 34)

# create the resistivity map
rho = [[1,80], 
      [2, 20],
      [3, 120],
      [4, 140]]

#create mesh

pg.show(mesh, data=rho, label=pg.unit('res'), showMesh=True)

In [ ]:
#simulate synthetic data

data = ert.simulate(mesh, scheme=scheme, res=rho, noiseLevel=1,
                    noiseAbs=1e-6, seed=1337)

pg.info(np.linalg.norm(data['err']), np.linalg.norm(data['rhoa']))
pg.info('Simulated data', data)
pg.info('The data contains:', data.dataMap().keys())

pg.info('Simulated rhoa (min/max)', min(data['rhoa']), max(data['rhoa']))
pg.info('Selected data noise %(min/max)', min(data['err'])*100, max(data['err'])*100)

In [ ]:
# data filter
# remove any data which is less or equal 0
#check minmax
data.remove(data['rhoa']<= 0)
pg.info('Filtered rhoa (min/max)', min(data['rhoa']), max(data['rhoa']))
data.save('EX3_synth.dat')

# You can take a look at the data
ert.show(data)

In [ ]:
# data inversion 
mgr = ert.ERTManager('EX3_synth.dat')
inv = mgr.invert(lam =20, verbose = True)
#np.testing.assert_approx_equal(mgr.inv.chi2(), 0.7, significant=1)

In [ ]:
mgr.showResultAndFit()
meshPD = pg.Mesh(mgr.paraDomain) # Save copy of para mesh for plotting later

In [ ]:
inversionDomain = pg.createGrid(x=np.linspace(start=-22, stop=22, num=33),
                                y=-pg.cat([0], pg.utils.grange(0.5, 8, n=5))[::-1],
                                marker=2)

In [ ]:
grid = pg.meshtools.appendTriangleBoundary(inversionDomain, marker=1,
                                           xbound=50, ybound=50)
pg.show(grid, markers=True)

In [ ]:
model = mgr.invert(data, mesh=grid, verbose=True)

In [ ]:
modelPD = mgr.paraModel(model)  # do the mapping
pg.show(mgr.paraDomain, modelPD, label='Model', cMap='Spectral_r',
        logScale=True, cMin=25, cMax=150)

pg.info('Inversion stopped with chi² = {0:.3}'.format(mgr.fw.chi2()))

fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True, sharey=True, figsize=(5.54,7))

pg.show(mesh, rho, ax=ax1, hold=True, cMap="Spectral_r", logScale=True,
        orientation="vertical", cMin=25, cMax=150)
pg.show(meshPD, inv, ax=ax2, hold=True, cMap="Spectral_r", logScale=True,
        orientation="vertical", cMin=25, cMax=150)
mgr.showResult(ax=ax3, cMin=25, cMax=150, orientation="vertical")

labels = ["True model", "Inversion unstructured mesh", "Inversion regular grid"]
for ax, label in zip([ax1, ax2, ax3], labels):
    ax.set_xlim(mgr.paraDomain.xmin(), mgr.paraDomain.xmax())
    ax.set_ylim(mgr.paraDomain.ymin(), mgr.paraDomain.ymax())
    ax.set_title(label)